In [ ]:
from test_configuration import plaid_client, firestore_client
from typing import Dict, Any, Union, List
import json
from datetime import datetime

from google.cloud.firestore import CollectionReference

from plaid.model.products import Products
from plaid import ApiException
from plaid.model.sandbox_public_token_create_request import SandboxPublicTokenCreateRequest
from plaid.model.account_balance import AccountBalance
from plaid.model.item_public_token_exchange_request import ItemPublicTokenExchangeRequest
from plaid.model.item_public_token_exchange_response import ItemPublicTokenExchangeResponse
from plaid.model.sandbox_public_token_create_response import SandboxPublicTokenCreateResponse

### Account Balance Get
from plaid.model.accounts_balance_get_request import AccountsBalanceGetRequest
from plaid.model.accounts_get_response import AccountsGetResponse

In [ ]:
def sandbox_public_token_create(institution_id: str) -> Dict[str, Any]:
    try:
        request = SandboxPublicTokenCreateRequest(
            institution_id=institution_id,
            initial_products=[Products('transactions')],
        )

        response: SandboxPublicTokenCreateResponse = plaid_client.sandbox_public_token_create(
            request
        )
        response_dict = response.to_dict()

        return response_dict
    except ApiException as e:
        exceptions: dict = json.loads(e.body)

        return exceptions

In [ ]:
def public_token_exchange(institution_id: Union[str, None]) -> Dict[str, Any]:
    # TODO: ^change the passed data to public_token in release

    if institution_id is None:
        return {'error_code': 404, 'error_message': 'institution_id was null'}

    try:
        # FOR SANDBOX TODO: comment for release
        public_token_response = sandbox_public_token_create(institution_id)
        print(public_token_response)
        public_token: Union[str, None] = public_token_response.get('public_token')
        print(f'public_token is {public_token}')    

        if public_token is None:
            return {'error_code': 404, 'error_message': 'public_token was None'}

        # Make Request to exchange public_token_to_access_token
        request = ItemPublicTokenExchangeRequest('public-sandbox-72c624b6-1307-499d-8f24-31d108820437')
        response: ItemPublicTokenExchangeResponse = plaid_client.item_public_token_exchange(request)
        response_dict = response.to_dict()
        print(f'exchange response dict: \n{response_dict}')

        return response_dict
    except ApiException as e:
        exceptions: dict = json.loads(e.body)
        return exceptions

In [ ]:
''' Retrieve real-time balance data

    The `/accounts/balance/get` endpoint returns the real-time balance for each of 
    an Item's accounts. While other endpoints may return a balance object, only 
    `/accounts/balance/get` forces the available and current balance fields to be 
    refreshed rather than cached. This endpoint can be used for existing Items 
    that were added via any of Plaid’s other products. This endpoint can be used 
    as long as Link has been initialized with any other product, balance itself 
    is not a product that can be used to initialize Link.
'''
def accounts_balance_get(access_token: str):
    try:
        request = AccountsBalanceGetRequest(access_token)
        response: AccountsGetResponse = plaid_client.accounts_balance_get(request)

        return response.to_dict()
    except ApiException as e:
        exceptions: dict = json.loads(e.body)
        return exceptions

In [ ]:
def update_accounts(uid: str, ins_id: str, accounts: List[dict]) -> Dict[str, Any]:
    try:
        user_doc = firestore_client.collection('users').document(uid)
        accounts_ref: CollectionReference = user_doc.collection('accounts')

        # Update Accounts in `Accounts` collections
        for account in accounts:
            print(f'Account: {account}')

            # Update Accounts Collections
            account_id: str = account.get('account_id')
            print(f'accountId: {account_id}')

            account_doc_ref = accounts_ref.document(account_id)
            account_snapshot = account_doc_ref.get()

            balances: Union[AccountBalance, None] = account.get('balances')
            print(f'Balances {balances}')

            data = {
                'account_id': account.get('account_id'),
                'balances': {
                    'available': balances.get('available'),
                    'current': balances.get('current'),
                    'iso_currency_code': balances.get('iso_currency_code'),
                    'limit': balances.get('limit'),
                    'unofficial_currency_code': balances.get('unofficial_currency_code'),
                },
                'mask': account.get('mask'),
                'name': account.get('name'),
                'official_name': account.get('official_name'),
                'subtype': str(account.get('subtype')),
                'type': str(account.get('type')),
                'verification_status': account.get('verification_status'),
                'account_last_synced_time': datetime.now(),
                'institution_id': ins_id,
                'account_connection_state': 'healthy'
            }

            if account_snapshot.exists:
                account_doc_ref.update(data)
            else:
                account_doc_ref.set(data)

            result = {
                'status': 200,
                'message': 'successfully updated accounts data',
            }

        return result
    except Exception as e:
        result = {
            'status': 404,
            'error_message': str(e),
        }

        return result

In [ ]:
def update_user_secrets(uid: str, access_token: str, institution_id: str) -> Dict[str, Any]:
    try:
        user_doc = firestore_client.collection('users').document(uid)
        secrets_collection: CollectionReference = user_doc.collection('secrets')
        plaid_secret_doc_ref = secrets_collection.document('plaid')
        plaid_secret_snapshot = plaid_secret_doc_ref.get() 

        new_access_token_dict = {institution_id: access_token}           
        
        if plaid_secret_snapshot.exists:
            secret_doc = plaid_secret_snapshot.to_dict()
            access_token_dict: Dict[str,str] = secret_doc.get('access_tokens')
            print(f'old access tokens: {access_token_dict}')

            access_token_dict[institution_id] = access_token
            print(f'new access tokens {access_token_dict}')

            plaid_secret_doc_ref.update({'access_tokens': access_token_dict})
        else:
            plaid_secret_doc_ref.create({'access_tokens': new_access_token_dict})

        result = {
            'status': 200,
            'message': 'Successfully updated access_tokens',
        }

        return result
    except Exception as e:
        result = {
            'status': 404,
            'error': str(e),
        }
        
        return result

In [ ]:
''' Link and Connect

    After a user connect a bank account and gets the `public_token`,
    this function is called to 
    1. Exchange `public_token` to `access_token`
    2. Using the `access_token`, call /accounts/balance/get
    3. Using the accounts info, update Firebase Firestore database
'''

def link_and_connect(request: Dict[str, Any]):
    # TODO: ^change the data type to flask.Request

    # TODO: for release
    # Parsing data from request to get `uid` and `public_token`
    data_dict: dict = json.loads(request.data)
    public_token: Union[str, None] = data.get('public_token')
    uid: Union[str, None] = data_dict.get('uid')
    institution_id: Union[str, None] = data_dict.get('institution_id')

    # # TODO: for sandbox
    # uid = request.get('uid') 
    # institution_id = request.get('institution_id')

    if (public_token and uid and institution_id) is None:
        return 'error_code=404, error_message=Please pass the right data in request'
        # return make_response(jsonify(error_code=404, error_message='Please pass the right data in request'), 404)

    # 1. Get access_token by using `item_public_token_exchange`
    token_exchange_response = public_token_exchange(institution_id)
    # TODO: change the data passed from `institution_id` to `public_token`
 
    access_token = token_exchange_response.get('access_token')
    print(f'Got access_token: {access_token}')

    # If access_token is None, return error
    if access_token is None:
        error_code = token_exchange_response.get('error_code')
        error_message = token_exchange_response.get('error_message')
        print(f'Error code: {error_code}')
        print(f'Error message: {error_message}')

        # # TODO: uncomment below for release
        # return make_response(jsonify(error_code=error_code, error_message=error_message), 404)

    # 2. Else, call /accounts/balance/get
    balances_get_response = accounts_balance_get(access_token)
    accounts: Union[List[Dict], None] = balances_get_response.get('accounts')
    print(f'balances got: {balances_get_response}')

    # If accounts is None, return error
    if accounts is None:
        error_code = balances_get_response.get('error_code')
        error_message = balances_get_response.get('error_message')
        print(f'Error code: {error_code}')
        print(f'Error message: {error_message}')

        # # TODO: uncomment below for release
        # return make_response(jsonify(error_code=error_code, error_message=error_message), 404)

    # 3. Update [Account] collection if hot accounts
    print(f'Got {len(accounts)} of accounts')
    update_account_result = update_accounts(uid, institution_id, accounts)

    # 4. Update user's secret keys with new access_token
    update_plaid_tokens_result = update_user_secrets(uid, access_token, institution_id)

    print(f'''
        update_account_result: {update_account_result}
        update_plaid_tokens_result: {update_plaid_tokens_result}
    ''')

    # # TODO: uncomment below for release
    # result = jsonify(
    #     update_account_result=update_account_result,
    #     update_plaid_tokens_result=update_plaid_tokens_result,
    # )

    # return make_response(result)

In [ ]:
data = {
    'institution_id': 'ins_10',
    'uid': 'uid',
}

link_and_connect(data)